# Generate DAG

## Import packages

In [94]:
import json
import operator
import numpy as np
import networkx as nx
import lucid.modelzoo.vision_models as models

## Functions to get general information of inception_v1 model

In [5]:
def get_layers(graph_nodes):
    '''
    Get all layers
    * input
        - graph_nodes: tensorflow graph nodes
    * output
        - layers: list of all layers
    '''
    layers = []
    for n in graph_nodes:
        node_name = n.name
        if node_name[-2:] == '_w':
            layer = node_name.split('_')[0]
            if layer not in layers:
                layers.append(layer)
    return layers

In [7]:
def get_channel_sizes(layer, weight_nodes):
    '''
    Get channel sizes
    * input
        - layer: the name of layer
        - weight_nodes: tensorflow nodes for all filters
    * output
        - channel_sizes: list of channel size for all pre-concatenated blocks
    '''
    
    channel_sizes = [get_shape_of_node(n)[0] for n in weight_nodes if layer in n.name and '_b' == n.name[-2:] and 'bottleneck' not in n.name]
    return channel_sizes

In [9]:
def get_shape_of_node(n):
    '''
    Get the shape of the tensorflow node
    * input
        - n: tensorflow node
    * output
        - tensor_shape: shape of n
    '''
    dims = n.attr['value'].tensor.tensor_shape.dim
    tensor_shape = [d.size for d in dims]
    return tensor_shape

## Functions to extract influences from I-matrices for a class

In [89]:
def extract_class_I_matrices(Is, all_layers, start_layer, end_layer, pred_class, verbose=True):
    '''
    Extract influences for a class from I-matrices
    * input
        - Is: I-matrices for all class
        - all_layers: list of all layers
        - start_layer: start layer (towards output)
        - end_layer: end layer (towards input)
        - pred_class: predicted class
    * output
        - Is_class: I-matrices for a class. a dictionary, where
            - key: layer name (e.g. 'mixed4d', 'mixed4d_1')
            - val: influences for given layer(key), class(argument of the function)
    '''

    # Get layers starting from the given layer to the input layer
    start_idx, end_idx = all_layers.index(start_layer), all_layers.index(end_layer)
    target_layers = all_layers[start_idx: end_idx - 1: -1]
    
    Is_class = {}
    
    for layer in target_layers:
        if verbose:
            print('\n({}) loading {}'.format(pred_class, layer), end='')
        Is_class[layer] = Is[layer][pred_class]
        for branch in [1, 2]:
            inner_layer = '{}_{}'.format(layer, branch)
            if verbose:
                print(',', inner_layer, end='')
            Is_class[inner_layer] = Is[inner_layer][pred_class]
    if verbose:
        print('\n')
    
    return Is_class

In [88]:
def load_I_matrices(all_layers, start_layer, end_layer, I_mat_dirpath, verbose=True):
    '''
    Load I-matrices for all layers
    * input
        - all_layers: list of all layers
        - start_layer: start layer (towards output)
        - end_layer: end layer (towards input)
        - I_mat_dirpath: directory path of I-matrices
    * output
        - Is: I-matrices for all class
    '''
    
    # Get layers starting from the given layer to the input layer
    start_idx, end_idx = all_layers.index(start_layer), all_layers.index(end_layer)
    target_layers = all_layers[start_idx: end_idx - 1: -1]

    # Load I matrices
    Is = {}
    for layer in target_layers:
        if verbose:
            print('\n(all) loading {}'.format(layer), end='')
        Is[layer] = load_inf_matrix(I_mat_dirpath, layer)
        for branch in [1, 2]:
            inner_layer = '{}_{}'.format(layer, branch)
            if verbose:
                print(',', inner_layer, end='')
            Is[inner_layer] = load_inf_matrix(I_mat_dirpath, inner_layer)
    if verbose:
        print('\n')    
    return Is

In [37]:
def load_inf_matrix(I_mat_dirpath, layer):
    '''
    Load I matrix for a layer
    * input
        - mat_dirpath: directory path of I-matrices
        - layer: layer name
    * output
        - I_mat: I-matrix of the given layer
    '''
    if I_mat_dirpath[-1] == '/':
        filepath = I_mat_dirpath + 'I_' + layer + '.json'
    else:
        filepath = I_mat_dirpath + '/I_' + layer + '.json'
        
    with open(filepath) as f:
        I_mat = json.load(f)
    
    return I_mat

## Functions to extract M-matrices information

In [159]:
def read_M(M_mat_dirpath, layer):
    M = np.loadtxt(M_mat_dirpath + 'M-' + layer + '.csv', delimiter=',', dtype=int)
    return M

In [226]:
def read_Ms(M_mat_dirpath, mixed_layers):
    Ms = {}
    for layer in mixed_layers:
        M = read_M(M_mat_dirpath, layer)
        Ms[layer] = M
    return Ms

## Functions to query the influence values

In [68]:
def get_branch(layer, channel, layer_channels):
    '''
    Get branch of the channel in the layer
    * input
        - layer: the name of layer
        - channel: channel in the layer
        - layer_channels: fragment sizes of the layer
    * output
        - branch: branch of the channel
    '''
    
    channels = layer_channels[:]
    for i in range(len(channels) - 1):
        channels[i + 1] += channels[i]
        
    branch = np.searchsorted(channels, channel, side='right')
    
    return branch

In [101]:
def avg_num_of_prevs_for_a_channel(layer, Is_class):
    '''
    Get the average number of previous channels connected to a channel in the given layer
    * input
        - layer: layer
        - Is_class: I-matrices for a class
    * output
        - num_avg: the average number of connections for a channel in the given layer
    '''
    
    num_of_channel_edges = []
    
    for channel, prev_inf_dict in enumerate(Is_class[layer]):
        # Get branch
        branch = get_branch(layer, channel, layer_fragment_sizes[layer])
        
        if branch in [0, 3]:
            num_of_channel_edges.append(len(prev_inf_dict))
            
    num_avg = int(np.average(num_of_channel_edges))
    
    return num_avg

## Functions to generate the graph

In [80]:
def node_name(layer, channel):
    return layer + '-' + str(channel)

In [109]:
def gen_full_graph(Is_class, G, mixed_layers):
    
    # Add edges into G from Is_class
    for layer_idx, layer in enumerate(mixed_layers[::-1][:-1]):
        # Get previous layer
        prev_layer = mixed_layers[::-1][layer_idx + 1]
        
        # Get the average number of edges for a channel
        avg_num_edges = avg_num_of_prevs_for_a_channel(layer, Is_class)
        
        # For all channels in layer
        for channel, prev_inf_dict in enumerate(Is_class[layer]):
            # Get source node
            src = node_name(layer, channel)
            
            # Get branch
            branch = get_branch(layer, channel, layer_fragment_sizes[layer])
            
            # If the channel is connected to a branch
            if branch in [1, 2]:
                # Get possible edge weights for the channel
                channel_edges = {}
                for prev_channel in prev_inf_dict:
                    prev_inf = prev_inf_dict[prev_channel]
                    
                    # Extract influence information for prev_channel
                    branch_layer = '{}_{}'.format(layer, branch)
                    prev_prev_inf_dict = Is_class[branch_layer][int(prev_channel)]
                    
                    for prev_prev_channel in prev_prev_inf_dict:
                        prev_prev_inf = prev_prev_inf_dict[prev_prev_channel]
                        if prev_prev_channel not in channel_edges:
                            channel_edges[prev_prev_channel] = []
                        channel_edges[prev_prev_channel].append(min(prev_inf, prev_prev_inf))
                        
                # Get only one weight for each channel and prev_prev channel
                for prev_prev_channel in channel_edges:
                    channel_edges[prev_prev_channel] = max(channel_edges[prev_prev_channel])
                
                # Get top (avg_num_edges) prev_prev_channels based on the edge weight
                top_prev_prevs_weights = sorted(channel_edges.items(), key=operator.itemgetter(1), reverse=True)
                top_prev_prevs_weights = top_prev_prevs_weights[:avg_num_edges]
                
                # Add edges from channel and top_prev_prev_channel
                for prev_prev_channel, weight in top_prev_prevs_weights:
                    tgt = node_name(prev_layer, prev_prev_channel)
                    G.add_edge(src, tgt, weight=weight)
            
            # If the channel is directly connected to the previous layer
            elif branch in [0, 3]:
                for prev_channel in prev_inf_dict:
                    # Add edge of (src, tgt-prev)
                    prev_inf = prev_inf_dict[prev_channel]
                    tgt = node_name(prev_layer, prev_channel)
                    G.add_edge(src, tgt, weight=prev_inf)

In [160]:
def init_dag(mixed_layers):
    dag = {}
    for layer in mixed_layers[::-1]:
        dag[layer] = []
    return dag

## Functions for pagerank

In [230]:
def get_personalization_dict(G, Ms, mixed_layers):
    
    personalization = {node: 1 for node in list(G.nodes)}

    for layer in mixed_layers[::-1]:
        M = Ms[layer]
        for channel in range(M.shape[-1]):
            node = layer + '-' + str(channel)
            if node in personalization:
                personalization[layer + '-' + str(channel)] = M[pred_class][channel]
    
    return personalization

## Functions for thresholding nodes, edges

In [130]:
def get_prob_mass(prob_mass_threshold, reverse_sorted_vals):
    prob_mass = 0
    threshold_cnt = 0
    while prob_mass < prob_mass_threshold:
        prob_mass += reverse_sorted_vals[threshold_cnt]
        threshold_cnt += 1
    threshold_val = reverse_sorted_vals[threshold_cnt]
    return threshold_cnt, threshold_val

In [152]:
def get_threshold(mixed_layers, pagerank, prob_mass_threshold=0.12):
    # Get threshold value
    pagerank_values = list(pagerank.values())
    sorted_pagerank_vals = sorted(pagerank_values, reverse=True)
    threshold_cnt, threshold_val = get_prob_mass(prob_mass_threshold, sorted_pagerank_vals)
    
    # Get thresholds
    thresholds = {}
    pagerank_sorted = sorted(pagerank.items(), key=operator.itemgetter(1))
    for layer in mixed_layers[::-1]:
        pageranks_layer = list(filter(lambda x: layer in x[0],  pagerank_sorted))
        pagerank_values_layer = list(map(lambda x: x[1], pageranks_layer))
        threshold = len(pagerank_values_layer) - np.searchsorted(np.array(pagerank_values_layer), 0.0007)
        thresholds[layer] = max(min(threshold, len(pagerank_values_layer) - 1), 0)

    return thresholds

In [205]:
def get_threshold_val(mixed_layers, pagerank, prob_mass_threshold=0.12):
    pagerank_values = list(pagerank.values())
    sorted_pagerank_vals = sorted(pagerank_values, reverse=True)
    threshold_cnt, threshold_val = get_prob_mass(prob_mass_threshold, sorted_pagerank_vals)
    
    return threshold_val

In [208]:
def get_thresholded_nodes(pagerank, threshold_val):
    thresholded_nodes = {}
    
    for node in pagerank:
        if pagerank[node] > threshold_val:
            thresholded_nodes[node] = pagerank[node]
        
    return thresholded_nodes

In [258]:
def get_thresholded_edges(mixed_layers, G):
    
    thresholded_edges = {}
    edge_checker = {}
    for layer in mixed_layers[::-1]:
        thresholded_edges[layer] = {}

    for node in G.nodes():
        for edge in G.edges(node):
            node1, node2 = edge
            if (node1 not in thresholded_nodes) or (node2 not in thresholded_nodes):
                continue

            layer, channel, prev_layer, prev_channel = parse_edge(edge, mixed_layers)
            if channel not in thresholded_edges[layer]:
                thresholded_edges[layer][channel] = []

            if (node1, node2) in edge_checker:
                continue
            elif (node2, node1) in edge_checker:
                continue
            edge_checker[(node1, node2)] = True

            thresholded_edges[layer][channel].append({
                'prev_channel': int(prev_channel),
                'inf:': G.get_edge_data(*edge)['weight']
            })
    return thresholded_edges

## Functions to generate DAG

In [259]:
def parse_edge(edge, mixed_layers):
    n1_layer, n1_channel = edge[0].split('-')
    n2_layer, n2_channel = edge[1].split('-')
    
    n1_idx, n2_idx = mixed_layers.index(n1_layer), mixed_layers.index(n2_layer)
    
    # If n1 is current layer, n2 is previous layer
    if n1_idx > n2_idx:
        layer, channel = n1_layer, n1_channel
        prev_layer, prev_channel = n2_layer, n2_channel
        
    # If n1 is previous layer, n1 is current layer
    else:
        layer, channel = n2_layer, n2_channel
        prev_layer, prev_channel = n1_layer, n1_channel
    
    return layer, channel, prev_layer, prev_channel

In [276]:
def gen_dag(mixed_layers, thresholded_nodes, thresholded_edges):
    dag = {}
    for layer in mixed_layers[::-1]:
        dag[layer] = []
        if layer == 'mixed3a':
            M = Ms[layer]
            for channel, cnt in enumerate(M[pred_class]):
                dag[layer].append({
                    'channel': int(channel),
                    'count': int(cnt),
                    'layer': layer,
                    'prev_channels': []
                })

    for node in G.nodes():
        for edge in G.edges(node):
            node1, node2 = edge
            if (node1 not in thresholded_nodes) or (node2 not in thresholded_nodes):
                continue

            layer, channel, prev_layer, prev_channel = parse_edge(edge, mixed_layers)
            M = Ms[layer]
            dag[layer].append({
                'channel': int(channel),
                'count': int(M[pred_class][int(channel)]),
                'layer': layer,
                'prev_channels': thresholded_edges[layer][channel]
            })
    return dag

## Get inception_v1 model infromation

In [261]:
data_dirpath = '/Users/haekyu/data/summit/'
imgnet_dirpath = data_dirpath
I_mat_dirpath = data_dirpath + 'I-matrices/'
M_mat_dirpath = data_dirpath + 'M-matrices/'
dag_dirpath = data_dirpath + 'dag/'

In [262]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()
nodes = googlenet.graph_def.node

In [263]:
all_layers = get_layers(nodes)
mixed_layers = [layer for layer in all_layers if 'mixed' in layer]
layer_fragment_sizes = {layer: get_channel_sizes(layer, nodes) for layer in mixed_layers}

In [264]:
with open(imgnet_dirpath + 'imagenet.json') as f:
    imgnet = json.load(f)

## Run for all classes

In [29]:
start_layer = 'mixed5b'
end_layer = 'mixed3a'

In [41]:
# Is = load_I_matrices(all_layers, start_layer, end_layer, I_mat_dirpath, verbose=True)


loading  mixed5b , mixed5b_1 , mixed5b_2 
loading  mixed5a , mixed5a_1 , mixed5a_2 
loading  mixed4e , mixed4e_1 , mixed4e_2 
loading  mixed4d , mixed4d_1 , mixed4d_2 
loading  mixed4c , mixed4c_1 , mixed4c_2 
loading  mixed4b , mixed4b_1 , mixed4b_2 
loading  mixed4a , mixed4a_1 , mixed4a_2 
loading  mixed3b , mixed3b_1 , mixed3b_2 
loading  mixed3a , mixed3a_1 , mixed3a_2 

In [277]:
num_class = 1000
for pred_class in range(num_class):
    if pred_class != 270:
        continue
    
    # Extract influence information for the pred_class
    # Is_class = extract_class_I_matrices(Is, all_layers, start_layer, end_layer, pred_class, verbose=False)
    
    # Initialize an undirected graph
    # G = nx.Graph()
    
    # Generate full graph
    # gen_full_graph(Is_class, G, mixed_layers)
    
    # Read M-matrices
    Ms = read_Ms(M_mat_dirpath, mixed_layers)
    
    # Personalized pagerank to filter nodes
    # personalization = get_personalization_dict(G, Ms, mixed_layers)
    # pagerank = nx.pagerank(G, personalization=personalization, weight='weight', alpha=0.90)
    
    # Thresolding
    threshold_val = get_threshold_val(mixed_layers, pagerank, prob_mass_threshold=0.12)
    thresholded_nodes = get_thresholded_nodes(pagerank, threshold_val)
    thresholded_edges = get_thresholded_edges(mixed_layers, G)
    
    # Generate dag in json format
    dag = gen_dag(mixed_layers, thresholded_nodes, thresholded_edges)
    
    # Save the graph into a file
    filename = dag_dirpath + 'pagerank/' + 'dag-{}.json'.format(pred_class)
    with open(filename, 'w') as f:
        json.dump(dag, f, indent=2)
                    
    print('class:', pred_class)          
    print(nx.info(G))

class: 270
Name: 
Type: Graph
Number of nodes: 5482
Number of edges: 350335
Average degree: 127.8128
